In [25]:
import csv
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [26]:
classes = ['Acer_Capillipes', 'Acer_Circinatum', 'Acer_Mono', 'Acer_Opalus', 'Acer_Palmatum', 'Acer_Pictum', 'Acer_Platanoids', 'Acer_Rubrum', 'Acer_Rufinerve', 'Acer_Saccharinum', 'Alnus_Cordata', 'Alnus_Maximowiczii', 'Alnus_Rubra', 'Alnus_Sieboldiana', 'Alnus_Viridis', 'Arundinaria_Simonii', 'Betula_Austrosinensis', 'Betula_Pendula', 'Callicarpa_Bodinieri', 'Castanea_Sativa', 'Celtis_Koraiensis', 'Cercis_Siliquastrum', 'Cornus_Chinensis', 'Cornus_Controversa', 'Cornus_Macrophylla', 'Cotinus_Coggygria', 'Crataegus_Monogyna', 'Cytisus_Battandieri', 'Eucalyptus_Glaucescens', 'Eucalyptus_Neglecta', 'Eucalyptus_Urnigera', 'Fagus_Sylvatica', 'Ginkgo_Biloba', 'Ilex_Aquifolium', 'Ilex_Cornuta', 'Liquidambar_Styraciflua', 'Liriodendron_Tulipifera', 'Lithocarpus_Cleistocarpus', 'Lithocarpus_Edulis', 'Magnolia_Heptapeta', 'Magnolia_Salicifolia', 'Morus_Nigra', 'Olea_Europaea', 'Phildelphus', 'Populus_Adenopoda', 'Populus_Grandidentata', 'Populus_Nigra', 'Prunus_Avium', 'Prunus_X_Shmittii', 'Pterocarya_Stenoptera', 'Quercus_Afares', 'Quercus_Agrifolia', 'Quercus_Alnifolia', 'Quercus_Brantii', 'Quercus_Canariensis', 'Quercus_Castaneifolia', 'Quercus_Cerris', 'Quercus_Chrysolepis', 'Quercus_Coccifera', 'Quercus_Coccinea', 'Quercus_Crassifolia', 'Quercus_Crassipes', 'Quercus_Dolicholepis', 'Quercus_Ellipsoidalis', 'Quercus_Greggii', 'Quercus_Hartwissiana', 'Quercus_Ilex', 'Quercus_Imbricaria', 'Quercus_Infectoria_sub', 'Quercus_Kewensis', 'Quercus_Nigra', 'Quercus_Palustris', 'Quercus_Phellos', 'Quercus_Phillyraeoides', 'Quercus_Pontica', 'Quercus_Pubescens', 'Quercus_Pyrenaica', 'Quercus_Rhysophylla', 'Quercus_Rubra', 'Quercus_Semecarpifolia', 'Quercus_Shumardii', 'Quercus_Suber', 'Quercus_Texana', 'Quercus_Trojana', 'Quercus_Variabilis', 'Quercus_Vulcanica', 'Quercus_x_Hispanica', 'Quercus_x_Turneri', 'Rhododendron_x_Russellianum', 'Salix_Fragilis', 'Salix_Intergra', 'Sorbus_Aria', 'Tilia_Oliveri', 'Tilia_Platyphyllos', 'Tilia_Tomentosa', 'Ulmus_Bergmanniana', 'Viburnum_Tinus', 'Viburnum_x_Rhytidophylloides', 'Zelkova_Serrata']
num_classes = len(classes)
num_labels = 990
print(len(classes))

99


In [27]:
train = pd.read_csv('train.csv')
parent_data = train.copy()
test = pd.read_csv('test.csv')
print(train.shape)

(990, 194)


In [28]:
def dense_to_one_hot(train_list):
    """Convert class labels from scalars to one-hot vectors."""
    labels_one_hot = np.zeros((len(train_list), num_classes))
    for i in xrange(len(train_list)):
    	labels_one_hot[i][train_list[i]] = 1 
    return labels_one_hot


In [29]:
train_id = train.pop('id')

In [30]:
train_labels = train.pop('species')
train_labels = LabelEncoder().fit(train_labels).transform(train_labels)
print train_labels.shape



(990,)


In [31]:
train_labels_one_hot = dense_to_one_hot(train_labels)
print (train_labels_one_hot.shape)

(990, 99)


In [32]:
train_data = StandardScaler().fit(train).transform(train)
print train_data.shape

(990, 192)


In [33]:
test = pd.read_csv('test.csv')
test_index = test.pop('id')
test_data = StandardScaler().fit(test).transform(test)
print test_data.shape

(594, 192)


In [34]:
sss = StratifiedShuffleSplit(train_labels,test_size=0.1, random_state=23)

for train_index, test_index in sss:
    train_data_2, valid_data = train_data[train_index], train_data[test_index]
    train_labels_2, valid_labels = train_labels[train_index], train_labels[test_index]
    train_labels_one_hot_2, valid_labels_one_hot = train_labels_one_hot[train_index], train_labels_one_hot[test_index]
print valid_data.shape, valid_labels_one_hot.shape, train_labels_2.shape

(99, 192) (99, 99) (891,)


In [35]:
batch_size = 128
features = 192
num_labels = 99
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [36]:
graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, 192))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_data, dtype= tf.float32)
    tf_test_dataset = tf.constant(test_data, dtype= tf.float32)

    # Variables.
    weights_1 = tf.Variable(
    tf.truncated_normal([192, 1024]))
    biases_1 = tf.Variable(tf.zeros([1024]))

    weights_2 = tf.Variable(
    tf.truncated_normal([1024,512]))
    biases_2 = tf.Variable(tf.zeros([512]))
    
    weights_3 = tf.Variable(
    tf.truncated_normal([512,num_labels]))
    biases_3 = tf.Variable(tf.zeros([num_labels]))
   
    
    def model(data):
        logits_1 = tf.nn.relu(tf.matmul(data, weights_1) + biases_1)
        
        logits_2 = tf.nn.relu(tf.matmul(logits_1, weights_2) + biases_2)
        logits_3 = tf.matmul(logits_2, weights_3) + biases_3
        return logits_3

    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    loss = loss + 0.01 * tf.nn.l2_loss(weights_2)+ 0.01 * tf.nn.l2_loss(weights_1)+ 0.01 * tf.nn.l2_loss(weights_3)

    # Optimizer.
    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 1200
data = pd.read_csv('test.csv')

index = data.pop('id')


with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    #train_writer = tf.train.SummaryWriter('/tmp/tf_log', sess.graph)
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_data_2[offset:(offset + batch_size), :]
        batch_labels = train_labels_one_hot_2[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
          print('Minibatch loss at step %d: %f' % (step, l))
          print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
          print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels_one_hot))
    test_pred = test_prediction.eval()
    test_pred = pd.DataFrame(test_pred,index=index,columns=classes)
    print test_pred.size
    fp = open('submission_nn_kernel.csv','w')
    fp.write(test_pred.to_csv())
  #print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    


Initialized
Minibatch loss at step 0: 12175.614258
Minibatch accuracy: 1.6%
Validation accuracy: 2.0%


ValueError: Cannot feed value of shape (123, 192) for Tensor u'Placeholder:0', which has shape '(128, 192)'